# Import a Molecules/Atom from cartesian coordinates:
**Task:** 
1. Create a Molecule object from its cartesian coordinates
2. Ask for its atomic and molecular properties

## [Water molecule](https://en.wikipedia.org/wiki/Properties_of_water) ($H_{2}O$):

<img 
src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/H2O_2D_labelled.svg/2560px-H2O_2D_labelled.svg.png"
alt="water" 
width="200" 
height="100" 
/>

### Properties:
- Number of molecules: 1
- Number of atoms: 3
- Atomic symbols: O, H, H
- Atomic masses [g/mol]: 
    - O: 15.999
    - H: 1.008
    - H: 1.008
- Molar mass [g/mol]: 18.015
- Cartesian coordinates [Angstrom]:
    ```XYZ
    O    0.00000    0.00000    0.00000
    H    0.58708    0.75754    0.00000
    H    0.58708   -0.75754    0.00000
    ```
- Bond distance [Angstrom]:
    - H-O: 0.9584 (95.84 pm)
    - H-H: 1.5151
- Bond angle [Degrees]:
    - H-O-H: 104.45
    - H-H-O: 37.78

In [3]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'src' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
# Import Molecule Class
from src.base_molecule import Molecule

In [5]:
# Creates a molecule. You can use a list, a dictionary (the key MUST be "atoms")
# or another molecule object (see below)

# 1 Option
water=[("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]
water_molecule = Molecule(water)

# 2 Option
water_dict = {"atoms": [("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]}
water_molecule = Molecule(water_dict)


In [6]:
## See initial molecule.

import py3Dmol

water_molecule = Molecule(water_dict)
water_molecule = water_molecule.translate(0, x=1.5, y=1.5, z=1.5)
bi_water = water_molecule.add_fragments(water_dict)


xyz_view = py3Dmol.view(width=300,height=200)
xyz_view.addModel(str(bi_water),'xyz')
xyz_view.setStyle({'stick':{}})

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [24]:
import numpy as np
from scipy.optimize import shgo
from pyscf import gto, scf


def optimization(x):
    """Funcion a optimizar. Primero se introduce la molécula de agua (water_dict), 
    luego, con la clase Molecule la trasladamos una cierta distancia (x=y=z=1.5) y luego
    agregamos el fragmento water_dict, para tener dos moléculas de agua separadas en 1.5 angs en todas direcciones. 

    Luego, a la molécula de 2 aguas, la trasladamos y rotamos en función de x, un 1-D array.
    A esa molécula, le calculanos la energía, siguiendo la base por defecto y RHF.  
    Parameters
    ----------
    x : [1-D Array, de 6 variables, en este caso]
        [variables x,y,z de traslación y los siguientes 3 de rotación]

    Returns
    -------
    []
        [Retorna la energía total del sistema, dado los parámetros en x]
    """
    

    water_molecule = Molecule(water_dict)
    water_molecule = water_molecule.translate(0, x=1.5, y=1.5, z=1.5)

    water_molecule_2 = Molecule(water_dict)
    water_molecule_2 = water_molecule_2.translate(0,x=x[0],y=x[1],z=x[2])
    water_molecule_2 = water_molecule_2.rotate(0,x[3],x[4],x[5])
    
    water_cluster = water_molecule.add_fragments(water_molecule_2)
    
    #Imprimir esto es caro... debería fuera imprimir sólo las geometrías finales
    #with open('water.xyz', 'w') as f:
    #    f.write(str(water_cluster))

    
    mol = gto.Mole()
    #mol.atom = "water.xyz"
    mol.fromstring(str(water_cluster))
    mol.build(basis = 'sto-3g')
    

    mf = scf.RHF(mol)
    mf.kernel()
    
    return mf.energy_tot()

water_dict = {"atoms": [("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]}
bounds = [(-1.5,1.5),(-1.5,1.5),(-2.5,2.5),(0,180),(0,90),(0,180)]

#aquí habría que probar con el método sobol, y también con otros métodos de optimización. 
minimos = shgo(optimization, bounds=bounds, sampling_method='simplicial')

print(minimos.xl, minimos.funl)



#Primera vez: demoró 24 minutos.
#Segunda vez: demoró 6 minutos


BasisNotFoundError: Invalid basis name TZ

In [96]:
#Dismiss candidates that are really similar to each other

for i in range(len(minimos.xl)):
        water_molecule = Molecule(water_dict)
        water_molecule = water_molecule.translate(0, x=1.5, y=1.5, z=1.5)

        water_molecule_2 = Molecule(water_dict)
        water_molecule_2 = water_molecule_2.translate(0,x=minimos.xl[i][0],y=minimos.xl[i][1],z=minimos.xl[i][2])
        water_molecule_2 = water_molecule_2.rotate(0,minimos.xl[i][3],minimos.xl[i][4],minimos.xl[i][5])

        water_cluster_optimized = water_molecule.add_fragments(water_molecule_2)

        if candidates: #si la lista no está vacía
                for k in range(len(candidates)):
                        if np.all(np.isclose(water_cluster_optimized.cartesian_coordinates, candidates_coord[k],atol=0.05)):
                                pass #valores muy cercanos
                        else:
                                candidates.append(water_cluster_optimized)
                                candidates_coord.append(water_cluster_optimized.cartesian_coordinates)
        else:
                candidates.append(water_cluster_optimized)
                candidates_coord.append(water_cluster_optimized.cartesian_coordinates)

In [95]:
#Save results on file

#Clean file
with open('water.xyz', 'w') as f:
                f.write(" ")
candidates=[]
candidates_coord=[]



for i in range(len(candidates)):
        mol = gto.Mole()
        mol.fromstring(str(candidates[i]))
        mol.build()

        mf = scf.RHF(mol)
        mf.kernel()
        energia = mf.energy_tot()


        with open('water.xyz', 'a') as f:

                f.write("---------------    Candidate %.0f    ---------------\n" % int(i+1))
                f.write(str(candidates[i]))
                f.write("Energy: %.6f \n" % mf.energy_tot())

In [97]:
## See Final molecule.

import py3Dmol

for i in range(len(candidates)):
    xyz_view = py3Dmol.view(width=300,height=200)
    xyz_view.addModel(str(candidates[i]),'xyz')
    xyz_view.setStyle({'stick':{}})

    xyz_view.zoomTo()
    xyz_view.show()



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
# Atomic Simulation Environment
# https://wiki.fysik.dtu.dk/ase/index.html
# !pip install --upgrade --user ase


# ChemML
# https://hachmannlab.github.io/chemml/index.html
# !pip install chemml

In [15]:
# NGLview
# https://github.com/nglviewer/nglview
# !pip install nglview

# ---------------------------------------
# pytraj 
# https://amber-md.github.io/pytraj/latest/index.html
# !pip install pytraj
